In [1]:
from os.path import join
import os
import pandas as pd
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from utilities import utils
import importlib
import numpy as np
importlib.reload(utils)
from utilities.utils import match_compare

In [2]:
base_dir = "D:/Thesis/Cobra/cobra/"
csv_folder = join(base_dir, 'data', 'tables')
neg_tab_dirs = sorted([f"{csv_folder}/{x}" for x
                       in os.listdir(csv_folder) \
                           if x.startswith('healthy')])
dst2 = pd.read_csv(join(csv_folder,'days_since_test2.csv'))

### Keys to add

In [51]:
df_keys = pd.read_csv(join(csv_folder, 'neg_pos.csv')).keys()
dfc_keys = pd.read_csv(join(csv_folder, 'neg_pos_clean.csv')).keys()
missing_keys = set(dfc_keys).difference(set(df_keys))
print(f"Missing columns in df_all: {missing_keys}")

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (4,5,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Missing columns in df_all: {'NumberOfSlices', 'RowSpacing', 'ColumnSpacing', 'days_since_test'}


C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Merge all negatives

## If you only want to update the clean dataframe, jump to create df_clean

In [3]:
df_neg_all = pd.read_csv(join(csv_folder, 'neg_all.csv'))

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_neg_all.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [23]:
df_neg_all[df_neg_all.SeriesInstanceUID.isnull()]

,SeriesInstanceUID,StudyInstanceUID,PatientID,AngioFlag,AcquisitionMatrix,AcquisitionContrast,AcquisitionDuration,dBdt,EchoTime,EchoTrainLength,...,SequenceName,ScanOptions,SeriesDescription,SoftwareVersions,SliceThickness,StudyPriorityID,PatientPosition.1,SpacingBetweenSlices,SecondEcho,VariableFlipAngleFlag
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_neg_all[df_neg_all.InstanceCreationDate.isnull()]

,SeriesInstanceUID,StudyInstanceUID,PatientID,AngioFlag,AcquisitionMatrix,AcquisitionContrast,AcquisitionDuration,dBdt,EchoTime,EchoTrainLength,...,SequenceName,ScanOptions,SeriesDescription,SoftwareVersions,SliceThickness,StudyPriorityID,PatientPosition.1,SpacingBetweenSlices,SecondEcho,VariableFlipAngleFlag
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315925,745dd7d98c52d643b6fd250fb3c8ca8b,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 250, 0]",NaN,NaN,0.0,333.00,137.0,...,*spcir3d1_274ns,"['IR', 'PFP', 'SAT1']",FLAIR cor,syngo MR B19,1.0,LOW,HFS,NaN,NaN,Y
315928,aa5f07e706901e9868311b62cbb76d38,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 256, 0]",NaN,NaN,0.0,2.92,1.0,...,*tfl3d1_ns,IR,MPR tra,syngo MR B19,1.0,LOW,HFS,NaN,NaN,N
315930,b6b0117a50349a9510472a4c183624a8,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 256, 0]",NaN,NaN,0.0,2.92,1.0,...,*tfl3d1_ns,IR,MPR cor +Gd,syngo MR B19,1.0,LOW,HFS,NaN,NaN,N
315932,d558cb734128ac0c344e017d211c4baa,450207a76fc81c60418f0cc94edc123f,fffe1330607e4941c51438ab0c0971b4,N,"[0, 256, 256, 0]",NaN,NaN,0.0,2.92,1.0,...,*tfl3d1_ns,IR,MPR cor,syngo MR B19,1.0,LOW,HFS,NaN,NaN,N


In [3]:
df_neg_list = [pd.read_csv(csv_path, encoding= 'unicode_escape') for csv_path in
               neg_tab_dirs]
df_all_neg_2019 = pd.concat(df_neg_list, axis=0, join="outer")
# In[Write it to a csv]
df_all_neg_2019.to_csv(f"{csv_folder}/neg_all.csv", index=False,
 header=True)

ipykernel_launcher:7: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.


# Merge negatives and positives

In [9]:
df_pos = utils.load_scan_csv(join(csv_folder, 'pos.csv'))

In [12]:
df_all = pd.concat([df_all_neg_2019, df_pos], axis=0, join="outer")
df_all.to_csv(f"{csv_folder}/neg_pos.csv", index=False,
 header=True)

In [13]:
df_all.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [14]:
df_pred = pd.read_csv(f"{csv_folder}/pred_seq.csv")

# Add Sequence Prediction

In [17]:
df_all = df_all.merge(df_pred.drop(labels=['PatientID', 'InstanceCreationDate'], axis=1), on='SeriesInstanceUID', how='inner')

In [20]:
df_directory = pd.read_csv(f"{csv_folder}/series_directories_sif.csv")
df_all_final = df_all.merge(df_directory, on='SeriesInstanceUID', how='inner')
df_all_final.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [21]:
df_all_final = df_all_final.rename(columns={'Directory': 'DirectorySIF'})

In [ ]:
def pixelspacing_convert(x, axis): 
    try:
        return x[axis]
    except:
        None
df_all['xPixelSpacing'] = df_all.PixelSpacing.map(lambda x: pixelspacing_convert(x, 0))
df_all['yPixelSpacing'] = df_all.PixelSpacing.map(lambda x: pixelspacing_convert(x, 1))

## Remember to add Positive column and DateTime

In [22]:
save_df_all=False
if save_df_all:
    df_all.to_csv(f"{csv_folder}/neg_pos.csv", index=False, header=True)

In [25]:
load_df_all=True
if load_df_all:
    df_all = utils.load_scan_csv(f"{csv_folder}/neg_pos.csv")

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
print(f"df_all contains {len(df_all)} volumes and\
 {df_all.PatientID.nunique()} patients")

df_all contains 328489 volumes and 24261 patients


## Clean

In [33]:
df_clean = df_all[~(df_all.Sequence=='other')]
print(f"after removing other sequences there are {len(df_clean)}\
 volumes and {df_clean.PatientID.nunique()} patients")

after removing other sequences there are 207027  volumes and 23307 patients


In [35]:
df_cclean = df_clean[~df_clean.InstanceCreationDate.isnull()]
print(f"after removing sequences with no date there are {len(df_cclean)} volumes\
 and {df_cclean.PatientID.nunique()} patients")

after removing sequences with no date there are 178578 volumes and 22434 patients


## Add Number of Slices

In [45]:
df_nos = pd.read_csv(join(csv_folder, 'neg_pos_clean_nos.csv'))
dfc = pd.merge(df_cclean, df_nos, how='left', on=['PatientID','SeriesInstanceUID'])

## Add days since test (days_since_test2)

In [50]:
df_dst = pd.read_csv(join(csv_folder, 'days_since_test2.csv'))
dfcc = pd.merge(dfc, df_dst, how='left', on=['PatientID','SeriesInstanceUID'])

## Create Rowspacing and Columnspacing columns

In [54]:
dfcc.PixelSpacing = dfcc.PixelSpacing.map(lambda x: utils.str_arr_converter(x))

In [58]:
def extract_from_arr(arr, pos): 
    try:
        return arr[pos] 
    except:
        return None
dfcc['RowSpacing'] = dfcc.PixelSpacing.map(lambda x: extract_from_arr(x, 0))
dfcc['ColumnSpacing'] = dfcc.PixelSpacing.map(lambda x: extract_from_arr(x, 1))

In [64]:
dfcc.to_csv(f"{csv_folder}/neg_pos_clean.csv", index=False, header=True)